In [11]:
import datetime
import hashlib
import re
import pandas as pd

from pathlib import Path

from orfdb import base, settings, annotation_loading
from orfdb.base import Assembly, Gene, Transcript, TranscriptExon, Exon, Orf, SequenceRegion, Dataset,\
                        Cds, CdsOrf, SequenceRegionXref, OrfXref,\
                        TranscriptOrf, Protein

from sqlalchemy import inspect, and_, update, func
from sqlalchemy_batch_inserts import enable_batch_inserting

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [6]:
settings.bigprot_version

'v0.9.4'

In [7]:
session = base.Session()

In [9]:
session.query(Orf, Assembly).join(Assembly, Assembly.id == Orf.assembly_id).groupby(Assembly.id).count()

AttributeError: 'Query' object has no attribute 'groupby'

In [17]:
session.query(Assembly).all()

[Assembly (#1): unmapped, unmapped, unmapped, -1,
 Assembly (#2): GCA_000001405.28_GRCh38.p13_assembly_report, chr1, CM000663.2, 248956422,
 Assembly (#3): GCA_000001405.28_GRCh38.p13_assembly_report, chr2, CM000664.2, 242193529,
 Assembly (#4): GCA_000001405.28_GRCh38.p13_assembly_report, chr3, CM000665.2, 198295559,
 Assembly (#5): GCA_000001405.28_GRCh38.p13_assembly_report, chr4, CM000666.2, 190214555,
 Assembly (#6): GCA_000001405.28_GRCh38.p13_assembly_report, chr5, CM000667.2, 181538259,
 Assembly (#7): GCA_000001405.28_GRCh38.p13_assembly_report, chr6, CM000668.2, 170805979,
 Assembly (#8): GCA_000001405.28_GRCh38.p13_assembly_report, chr7, CM000669.2, 159345973,
 Assembly (#9): GCA_000001405.28_GRCh38.p13_assembly_report, chr8, CM000670.2, 145138636,
 Assembly (#10): GCA_000001405.28_GRCh38.p13_assembly_report, chr9, CM000671.2, 138394717,
 Assembly (#11): GCA_000001405.28_GRCh38.p13_assembly_report, chr10, CM000672.2, 133797422,
 Assembly (#12): GCA_000001405.28_GRCh38.p13_as

In [15]:
results = (
    session.query(Assembly.genbank_accession, func.count(Orf.id))
    .join(Orf, Orf.assembly_id == Assembly.id)
    .group_by(Assembly.id)
    .all()
)

for i, (assembly, count_transcripts) in enumerate(results):
    print(i, assembly, count_transcripts)

0 CM000663.2 1131148
1 CM000664.2 924481
2 CM000665.2 687204
3 CM000666.2 493917
4 CM000667.2 577369
5 CM000668.2 614259
6 CM000669.2 612386
7 CM000670.2 483683
8 CM000671.2 527246
9 CM000672.2 491415
10 CM000673.2 637789
11 CM000674.2 587799
12 CM000675.2 267943
13 CM000676.2 468876
14 CM000677.2 448163
15 CM000678.2 532178
16 CM000679.2 599324
17 CM000680.2 231321
18 CM000681.2 562930
19 CM000682.2 294433
20 CM000683.2 167453
21 CM000684.2 331279
22 CM000685.2 388310
23 CM000686.2 112034
24 KI270711.1 202
25 KI270713.1 51
26 KI270721.1 32
27 GL000009.2 92
28 GL000194.1 150
29 KI270726.1 19
30 KI270727.1 388
31 KI270728.1 421
32 GL000205.2 2
33 KI270731.1 148
34 KI270733.1 8
35 KI270734.1 375
36 GL000195.1 69
37 GL000213.1 140
38 GL000218.1 146
39 GL000219.1 6
40 GL000220.1 8
41 KI270442.1 4
42 KI270750.1 1
43 KN196472.1 251
44 KN196473.1 465
45 KN196474.1 272
46 KN538360.1 893
47 KN538361.1 1420
48 KQ031383.1 2488
49 KZ208906.1 750
50 KQ458382.1 56
51 KQ458383.1 1738
52 KQ458384.1 60

In [13]:
a[2].genbank_accession

'CM000664.2'

In [7]:
t.transcript_idx_str

'11874_14409_+_2_353;108;1188_11874;12613;13221'

In [5]:
t2 = session.query(Transcript).filter(Transcript.refseq_id != '').first()

In [6]:
t2.transcript_idx_str

'127588411_127591700_+_8_154;80;109;71;125;487_127588411;127589083;127589485;127590066;127590963;127591213'

In [9]:
session.query(Transcript).filter(Transcript.transcript_idx_str == '138875400_138935034_+_6_197;93;52;92;80;152;92;156;200;151;110;181;105;134;1232_138875400;138880195;138880681;138881045;138886212;138887490;138904349;138917742;138924510;138925256;138929246;138930473;138930830;138932578;138933802').all()

[]

In [8]:
t2

Transcript: (1, ENST00000000233.10, NM_001662.4)+:127588411-127591700

In [16]:
session.query(Transcript).count()

444772

In [4]:
settings.bigprot_version

'v0.9.3'

In [4]:
ls $settings.bigprot_directory/v0.9.3/

README.md
orf_idx_vtx_map_240604.pkl
orfset_v0.9.2_full_minlen_15_maxlen_999999999_dashboard_orfs_with_bigprot_mapping.csv.gz
orfset_v0.9.3_full_minlen_15_maxlen_999999999.find_orfs.log
orfset_v0.9.3_full_minlen_15_maxlen_999999999.find_orfs.log.gz
orfset_v0.9.3_full_minlen_15_maxlen_999999999_cds_orfs.csv.gz
orfset_v0.9.3_full_minlen_15_maxlen_999999999_orfs.csv.gz
orfset_v0.9.3_full_minlen_15_maxlen_999999999_orfs_supplemental.csv.gz
orfset_v0.9.3_full_minlen_15_maxlen_999999999_transcript_orfs.csv.gz
orfset_v0.9.3_full_minlen_15_maxlen_999999999_xrefs_exported.parquet


In [20]:
from pathlib import Path
import re
from packaging.version import Version

def get_highest_version_folder(directory):
    """
    Finds the folder with the highest version number in the specified directory.
    
    Args:
        directory (str or Path): Path to the directory to scan.
    
    Returns:
        Path or None: The folder with the highest version, or None if no valid version folders are found.
    """
    directory = Path(directory)
    if not directory.is_dir():
        raise ValueError(f"{directory} is not a valid directory.")
    
    version_folders = {}
    version_pattern = re.compile(r"^v\d+(\.\d+)*$")  # Matches version-like patterns (e.g., 1.0.0)

    for folder in directory.iterdir():
        if folder.is_dir() and version_pattern.match(folder.name):
            try:
                version_folders[folder] = Version(folder.name)
            except ValueError:
                pass  # Ignore folders that don't follow proper versioning

    if version_folders:
        return max(version_folders, key=version_folders.get)
    return None

In [21]:
ls $settings.gencode_directory/v42

GCA_000001405.28_GRCh38.p13_assembly_report.txt
Ribo-seq_ORFs.bb
Ribo-seq_ORFs.bed
gencode.v42.2wayconspseudos.gff3.gz
gencode.v42.chr_patch_hapl_scaff.annotation.expanded.gff3
gencode.v42.chr_patch_hapl_scaff.annotation.gff3.gz
gencode.v42.long_noncoding_RNAs.expanded.gff3.gz
gencode.v42.long_noncoding_RNAs.gff3.gz
gencode.v42.metadata.Annotation_remark.gz
gencode.v42.metadata.EntrezGene.gz
gencode.v42.metadata.Exon_supporting_feature.gz
gencode.v42.metadata.Gene_source.gz
gencode.v42.metadata.HGNC.gz
gencode.v42.metadata.PDB.gz
gencode.v42.metadata.PolyA_feature.gz
gencode.v42.metadata.Pubmed_id.gz
gencode.v42.metadata.RefSeq.gz
gencode.v42.metadata.SwissProt.gz
gencode.v42.metadata.TrEMBL.gz
gencode.v42.metadata.Transcript_source.gz
gencode.v42.metadata.Transcript_supporting_feature.gz
gencode.v42.polyAs.gff3.gz
gencode.v42.tRNAs.gff3.gz
gencode.v42.transcripts.fa.gz


In [22]:
highest_version_folder = get_highest_version_folder(settings.bigprot_directory)

In [23]:
version = highest_version_folder.name

In [24]:
required_files = [
    f'orfset_{version}_full_minlen_15_maxlen_999999999_orfs.csv.gz',
    f'orfset_{version}_full_minlen_15_maxlen_999999999_transcript_orfs.csv.gz',
    f'orfset_{version}_full_minlen_15_maxlen_999999999_cds_orfs.csv.gz'
]

files_exist = all(
    highest_version_folder.joinpath(filename).exists() 
    for filename in required_files
)

if files_exist:
    print('a')

a


In [25]:
tx_path = highest_version_folder.joinpath(f'orfset_{version}_full_minlen_15_maxlen_999999999_transcript_orfs.csv.gz')

In [26]:
import gzip

In [28]:
vals

['{}', 'big_prot', '2821298', '9_113286750_113286809_+_113286750_60', '292208']

In [30]:
transcript_ids = set()

with gzip.open(tx_path, 'rt') as gz_file:  # 'rt' mode is for reading text
    for i, line in enumerate(gz_file):
        vals = line.rstrip('\n').split(',')
        if i < 10:
            print(vals[4])
        transcript_ids.add(vals[3])

transcript_orf.transcript_id
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [36]:
len(transcript_ids)

470515